Group Name: AG xx.

Student Name (Student ID):

1. Zhao Yufan (xxxxxxx)

2. xxxx xxxxx (xxxxxxx)

3. xxxx xxxxx (xxxxxxx)

# Question 2

## Introduction to question 2

In the second question of this assignment, we will explore the use of local search in genome assembly.

We will use local search to assemble (construct) a large part of the nucleotide sequence of the monkeypox virus, which has been downloaded from the National Center for Biotechnology Information in the United States. Please note that no additional or specialized knowledge of biology or bioinformatics is required for this assignment. (Actually, the technical specifics of bioinformatics have been adapted and simplified for the purposes of this computer science assignment, so if you are a biologist, please do not apply preexisting knowledge to solve the problem. Furthermore, you should not attempt to search up the genome on genomic databases to "guess" the actual sequence, since we are more interested in your coding methodology rather than your attempts at reproducing a known sequence.)

This is an introductory computer science assignment and not a bioinformatics assignment; we are simply using bioinformatics as a use case to illustrate the applicability of local search to the natural sciences. Therefore, no knowledge of bioinformatics is assumed or required. In the paragraphs that follow, I will give a short crash course which will cover all the domain knowledge you will need to know in order to tackle this problem.  

For technical reasons, when we analyze the nucleotide sequence (genome) of a virus, we usually cannot “read” it in one fell swoop. We have to read the genome in parts, because the genome is usually too long for the machine to read in a single sitting. To simplify things, a “read” is a single view of part of the genome; think of it as a SUBSTRING, a partial view of the whole genome. After we have generated multiple reads of a genome, we then have to “stitch”, or combine, the different reads of the genome together. This process of stitching up reads of a genome into the final sequence is known as genome assembly. However, the different reads of the genome cannot just be concatenated like usual string concatenation. It’s not a situation where you have one read, “Hello”, and another read, “World”, and all you need to do is concatenate both strings together to make “Hello World”. Among other reasons, there are two major reasons why you can’t do so:

1. You do not know which read came first. The reads are not ordered. How do you know “Hello” came after “World”? The answer is that you don’t. Imagine how complicated this situation might be if you had more than two reads. (This is indeed our situation, where we have $n$ reads, and $n>>2$.)

2. One read may contain a substring contained in another read. Specifically, without loss of generality, part of the ending $x$ characters of a read (i.e., suffix) might also be found in the starting $x$ positions (i.e., prefix) of another read.

- A computer scientist usually creates opportunities from problems. While this may be a “problem” in that you just can’t concatenate two strings blindly, the fact that strings contain shared “substrings” is actually a very helpful clue that you can use to “join” strings together. 

- Note that the choice of the value of $x$ could be a hyperparameter decided by the computer scientist.

## Your tasks

In this part of the assignment, you will work with (simulated) reads that I have generated from the nucleotide sequence of the monkeypox virus. In reality, bioinformatics is far more complicated, but here we will work with a simplified situation. Your task is to examine the reads that I have provided for you, and from there “infer” the nucleotide sequence that might have produced those reads. 

The reads are provided in the csv file `data.csv` which simply provides a list of unique strings. Note that you should NOT assume any particular ordering of the strings in this dataframe. In fact, the strings have already been shuffled randomly. 

NOTE: You are not allowed to use `pandas` or any other libraries apart from the Python STL to load the csv file.

### Task A (3 marks): 

Create a directed graph. The nodes in the graph are the strings in the list of reads. An edge should be drawn FROM read A TO read B if and only if a suffix (of length $x$) of read A is also a prefix (obviously, also of length $x$) of read B. For the purposes of the assignment, limit the value of $x$ to between 5 and 30, both inclusive. That is, to be clear, $5\leq x\leq 30$. The weight of an edge between read A and read B should be the NEGATED value of $x$, i.e. $-x$. 

In your Jupyter notebook, please report the number of edges in your graph. Provide a barplot or histogram which shows the number of edges with different weights or weight categories. In this task, you are free to use plotting libraries such as `matplotlib` or `seaborn` to plot this graph.

As an example, if read A is "TACTAGT" and read B is "TAGTCCCCT", then an edge is drawn FROM read A TO read B (i.e., $A \rightarrow B$) with weight of $-4$. This is because the 4-suffix "TAGT" is also the 4-prefix of read B; in other words, the last 4 characters of read A (a substring of length 4) overlap with the first 4 characters of read B (a substring of length 4).

### Task B (7 marks): 

From Task A, you now have a graph which shows connections between reads based on how they overlap, in theory you could draw a path through the graph and thereby derive the full sequence (genome).

Task B asks you to use local search method(s) to determine a path through this directed graph of strings. 

- You are expected to use simulated annealing and tune the relevant configuration settings and hyperparameters. The minimum requirement is to implement simulated annealing.

- Explain tha rationale behind the choice of scheduling strategy and parameters.

- However, you may also explore other search methods in addition to simulated annealing. Marks will be awarded for effort.

Note the following constraints:

1. The path has to go through each and every vertex exactly once. For computer scientists, this constraint is reminiscent of the "Traveling Salesman's Problem", except that unlike TSP, we should not need to go back to the starting vertex again. 

2. For the purposes of neighbor generation / action selection at each node, bear in mind that a path through the graph which minimizes the total number of nucleotides in the assembled sequence is the preferred path. To state that another way, the assembled sequence should be derived from a path that goes through EACH and EVERY vertex exactly once, however we want this assembled sequence to be AS SHORT AS POSSIBLE.

3. You are not given the starting (source/origin) or ending (destination) vertex.

4. For avoidance of ambiguity, no cycles are allowed. You must not visit a vertex more than once.

5. You are not allowed to use any libraries apart from the Python Standard Library.
No import statements which import libraries outside of the Python STL should be found within your answer for Task B.

Please remember to report the assembled sequence that you obtain. Although it would be great if you can come up with a good sequence, please feel reassured that we are more interested in your APPROACH to the problem, and so you can potentially get a reasonable score on this task even if your solution is "wrong". It is the process, rather than the result, which matters more.

In [1]:
# Problem Class
class Problem:
    """The abstract class for a formal problem. A new domain subclasses this,
    overriding `actions` and `results`, and perhaps other methods.
    The default heuristic is 0 and the default action cost is 1 for all states.
    When you create an instance of a subclass, specify `initial`, and `goal` states 
    (or give an `is_goal` method) and perhaps other keyword args for the subclass."""

    def __init__(self, initial=None, goal=None, **kwds): 
        self.__dict__.update(initial=initial, goal=goal, **kwds) 
        
    def actions(self, state):        raise NotImplementedError
    def result(self, state, action): raise NotImplementedError
    def is_goal(self, state):        return state == self.goal
    def action_cost(self, s, a, s1): return 1
    def h(self, node):               return 0
    
    def __str__(self):
        return '{}({!r}, {!r})'.format(
            type(self).__name__, self.initial, self.goal)

In [ ]:
# Use the following Node class to generate search tree
import math
class Node:
    "A Node in a search tree."
    def __init__(self, state, parent=None, action=None, path_cost=0):
        self.__dict__.update(state=state, parent=parent, action=action, path_cost=path_cost)

    def __repr__(self): return '<{}>'.format(self.state)
    def __len__(self): return 0 if self.parent is None else (1 + len(self.parent))
    def __lt__(self, other): return self.path_cost < other.path_cost 

In [ ]:
# Code to generate neighbours, value of states, etc.
class TSP(Problem):
    #Implement TSP class here
    pass

$\textbf{My understanding of the task}$:

1. We have a biological virus with a really really long DNA, that is too long to read from beginning to end.
2. We have read short portions of it multiple times, and we hope to piece them together.
3. The short reads are given in the data.csv file.
4. Suppose we have 2 reads X and Y. If the characters towards the end of read X matches those at the beginning of read Y, there is a good chance that the X and Y were together in the original DNA.

$\textbf{Toy Example}$:
rawData2 =  [['0' , 'read_0' , 'TCGATCTG'] , ['1' , 'read_1' , 'TCTGA'] , ['2' , 'read_2' , 'TGAGA'] , ['3' , 'read_3' , 'ATCGA']]

A possible DNA would be ATCGATCTGAGA, where we joined (read_3) - (read_0) - (read_1) - (read_2) , because there is substantial overlap between each read. The bigger the overlap, the better.

$\textbf{What we need to do for this question}$:

Repeat the toy example, but for rawData = data.csv

I have written some helper functions that I think will be useful:


$\textbf{SufPreCompare(string1 = '' , string2 = '')}$ does the following:
    Given 2 strings string1 and string2, returns a tuple (a,b), where:
        a is the length of overlap of suffix of string 1 and prefix of string 2
        b is the length of overlap of suffix of string 2 and prefix of string 1

In [2]:
def SufPreCompare(string1 = '' , string2 = ''):
    
    def StrCompare(string1, string2):
        maxindex = min(len(string1) , len(string2),30)

            
        counter = maxindex
        
        while counter > 0:
            
            str1 = string1[len(string1) - counter : ]
            str2 = string2[0 : counter]
            
            if str1 == str2:
                return counter
            else:
                counter = counter - 1
        return counter
    
    a = StrCompare(string1, string2)
    b = StrCompare(string2, string1)
    return (a,b)

In [3]:
SufPreCompare(string1 = 'TCGATCTG' , string2 = 'TCTGA')

(4, 0)

Explanation: The last 4 letters of string1 matches the first 4 letters of string2. The last 0 letters of string2 matches the first 0 letters of string1.

In [4]:
SufPreCompare(string1 = 'TCGATCTG' , string2 = 'TCTGATCGAT')

(4, 5)

Explanation: The last 4 letters of string1 matches the first 4 letters of string2. The last 5 letters of string2 matches the first 5 letters of string1.

$\textbf{read_line_by_lineRaw(filename = '')}$ does the following:
    Open up a file stored in your computed called (filename), and returns its contents as a list. It is intended to be used on data.csv
    
The last line rawData.pop(0) removes the header. The output is the row that was removed from our data.

In [15]:
#read_line_by_lineRaw takes a csv file and returns its rows as a list    
def read_line_by_lineRaw(filename = ''):
    output = []
    with open(filename) as f:
        for line in f:
            a = line.split(',')
            output.append(a)
    return output

$\textbf{You need to upload the data.csv file to the jupyter notebook environment for the next cell to work:}$

In [16]:
#rawData is a list of reads, with each row being:
#   ['row_count' , 'read_count' , 'read_string']
rawData = read_line_by_lineRaw('data.csv')

#remove the header row
rawData.pop(0)

['', 'read_index', 'read_sequence\n']

 There is a possibility that a particular read X gives a substring of another read Y
 We eliminate this possibility by doing the following:
    
1. Compare every possible pair of reads.
2. If one of them is a substring of the other, change read to 'x'.
3. Write a file ProcessedData.txt with 'x' reads intact.
4. Write another file ProcessedData2.txt  with 'x' reads removed.
    
After manually comparing the number of lines in ProcessedData.txt and ProcessedData2.txt, we realise that
None of the reads are sub-reads of another read.
So we can use rawData as it is. The code below does the above.
Since this is a one-time check, the manual check above is fine.

In [9]:
for i in range(len(rawData)):
    for j in [x for x in range(len(rawData)) if x != i]:
        if rawData[i][2] in rawData[j][2]:
            rawData[i] = [rawData[i][0] , rawData[i][1] , 'x']

with open('ProcessedData.txt' , 'w') as g:
    for i in range(len(rawData)):
        if rawData[2] != 'x':
            g.write(f'{rawData[i][0]},{rawData[i][1]},{rawData[i][2]}\n')
        else:
            continue

with open('ProcessedData2.txt' , 'w') as g:
    for i in range(len(rawData)):
        g.write(f'{rawData[i][0]},{rawData[i][1]},{rawData[i][2]}\n')

Next, we want to produce a directed graph with the following as vertices and directed edge labels:
1. Vertices are the reads
2. Given 2 vertices A and B, the edge from A to B has edge label  -(length of overlap of end of A and beginning of B) 

$\textbf{graphEdgeConstructor(m , n , dataset)}$ does the following: 
Given two reads m and n in a dataset (which should be the list returned by read_line_by_lineRaw(filename = ''), not the original file data.csv), returns a tuple of tuples ((m,n) , (x,y)), where:
1. x is the overlap between the end of read m and beginning of read n
2. y is the overlap between the end of read n and beginning of read m

$\textbf{contructEdges(dataset)}$ does the following:
1. Scan through the entire dataset.
2. For pairs of reads (m,n) with non-zero overlaps (x,y) as defined above in graphEdgeConstructor(m , n , dataset), put all of them into a list.

In [12]:
def graphEdgeConstructor(m , n , dataset):
    comparison = SufPreCompare(dataset[m][2] , dataset[n][2])
    return ((m,n) , comparison)

def contructEdges(dataset):
    output = []
    test4 = len(dataset)
    for i in range(test4):
        for j in range(i+1,test4):
            temp = graphEdgeConstructor(i , j, dataset)
            if temp[1] != (0,0):
                output.append(temp)

    return output

We test these functions on the Toy Example:

In [21]:
rawData2 = [['0','read_0','TCGATCTG'],['1','read_1','TCTGA'],['2','read_2','TGAGA'],['3','read_3','ATCGA']]
rawDataMini = rawData[0:300]
#rawDataMini is the first 300 rows of data.csv in list format

contructEdges(rawData2)

[((0, 1), (4, 0)),
 ((0, 2), (2, 0)),
 ((0, 3), (0, 4)),
 ((1, 2), (3, 0)),
 ((1, 3), (1, 0)),
 ((2, 3), (1, 0))]

$\textbf{Explanation}:$ We see that the biggest numbers that appear on the right column of tuple are 4, 4, and 3 in rows 0, 2, and 3. This suggests the following 'joining-up' of our DNA sequence:

1. row 0: This means there are 4 overlapping letters at the end of read_0 and beginning of read_1
2. row 2: This means there are 4 overlapping letters at the end of read_3 and beginning of read_0
3. row 3: This means there are 3 overlapping letters at the end of read_1 and beginning of read_2

This suggests our DNA should be (read_3) - (read_0) - (read_1) - (read_2). Hence:

DNA = 'ATCGATCTGAGA'

In [19]:
contructEdges(rawData)

[]

In [20]:
contructEdges(rawDataMini)

[]

There seems to be no overlaps between any pair of reads in all of rawData (list of data.csv)? What is going on?